In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
from scipy.stats import norm,multivariate_normal

import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression



In [2]:
# logging_index
# eval_index

# logging_policy 1
# eval_policy 1
# logging_xshift 1
# eval_xshift 1
# first_component 1
# logging_xy 1
# eval_xy 1
# logging_reward 1
# eval_reward 1
# logging_label 1
# eval_reward 1



In [3]:
# box 1

# xy
# label
# fc

data = pd.read_csv('optdigits.csv',header = None)
from sklearn.utils import shuffle
data = shuffle(data)


label = np.array(data[64]) - 1
xy = np.array(data.drop([64],axis = 1))
n_class = 10
n_dim = xy.shape[1]

train_xy = xy[:int(len(xy)*0.7)]
test_xy = xy[int(len(xy)*0.7):]

train_label = label[:int(len(xy)*0.7)]
test_label = label[int(len(xy)*0.7):]




pca_model = PCA(n_components=1).fit(xy)
fc = pca_model.transform(xy)

train_fc = fc[:int(len(xy)*0.7)]
test_fc = fc[int(len(xy)*0.7):]

train_size = int(len(xy)*0.7)



In [58]:
# box 2

logging_a = 0.8
logging_b = 0.3

eval_a = 1
eval_b = 0.01

def get_distribution(fc,a,b,pca_model=None,test = False):
    if test:
        pdf = np.ones(len(fc)).reshape(len(fc),1)
        return pdf/sum(pdf), 0, 0
    else:
        m = np.min(fc)
        m_bar = np.mean(fc)
        mean = m + (m_bar-m)/a
        std = np.sqrt((m_bar - m)/b)
        pdf = norm(loc = mean,scale = std).pdf(fc)
    #     pdf = np.ones(len(fc))
    return pdf/sum(pdf), mean, std








In [59]:
# # box 3
# # logging policy, eval_policy
# logging_policy_interval = [0.5,0.8]
# eval_policy_interval = [0.8,0.9]

def generate_policy_soften(alpha,beta):
    clf_policy = LogisticRegression(random_state=0).fit(train_xy[:int(train_size*0.1)], train_label[:int(train_size*0.1)])
    predict_label = clf_policy.predict(xy)
    logging_policy = np.zeros((len(xy),n_class))

    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
        u = np.random.uniform(-0.5,0.5)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = alpha + u*beta
            else:
                tmp[0,t] = (1-alpha - u*beta)/(n_class-1)
        logging_policy[i] = tmp



    clf_policy = LogisticRegression(random_state=1).fit(train_xy, train_label)
    predict_label = clf_policy.predict(xy)
    eval_policy = np.zeros((len(xy),n_class))
    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
#         u = np.random.uniform(-0.05,0.05)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 0.9
            else:
                tmp[0,t] = (1-0.9)/(n_class-1)
        eval_policy[i] = tmp
    return logging_policy, eval_policy
    

In [60]:
# # box 3
# # logging policy, eval_policy
# logging_policy_interval = [0.5,0.8]
# eval_policy_interval = [0.8,0.9]

def generate_policy_adv(alpha,beta):
    clf_policy = LogisticRegression(random_state=0).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    logging_policy = np.zeros((len(xy),n_class))

    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
        u = np.random.uniform(-0.5,0.5)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 1- alpha - u*beta
            else:
                tmp[0,t] = (alpha + u*beta)/(n_class-1)
        logging_policy[i] = tmp



    clf_policy = LogisticRegression(random_state=1).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    eval_policy = np.zeros((len(xy),n_class))
    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
#         u = np.random.uniform(-0.05,0.05)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 0.9
            else:
                tmp[0,t] = (1-0.9)/(n_class-1)
        eval_policy[i] = tmp
    return logging_policy, eval_policy
    

In [61]:
# sample data
# just sample index and everything can be refered from index.
def take_action(policy):
    action = random.choices([t for t in range(n_class)],policy)[0]
    prob = policy[action]
    return action, prob


def sample_logging_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy):
    logging_index = np.zeros(size,np.int)
    logging_action = np.zeros(size,np.int)
    logging_reward = np.zeros(size,np.int)
    index_list = [i for i in range(len(xy))]
    
    for i in range(size):
        t_index = random.choices(index_list,weights=pdf[:train_size])[0]
        t_index = t_index
        logging_index[i] = t_index
        
        action,_ = take_action(logging_policy[t_index])
        logging_action[i] = action
        
        logging_reward[i] = int(action == label[t_index])
    
    return logging_index, logging_action, logging_reward




def sample_eval_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy):
    eval_index = np.zeros(size,np.int)
    eval_action = np.zeros(size,np.int)
    eval_reward = np.zeros(size,np.int)
    index_list = [i for i in range(len(xy))]
    
    for i in range(size):
        t_index = random.choices(index_list,weights=np.ones(len(xy)))[0]
        t_index = t_index  + train_size
        eval_index[i] = t_index  
        
        action,_ = take_action(logging_policy[t_index])
        eval_action[i] = action
        
        eval_reward[i] = int(action == label[t_index-train_size])
    
    return eval_index , eval_action, eval_reward




        



In [62]:
def p_xshift(train_x,test_x):
    from sklearn.linear_model import LogisticRegression
    X = np.concatenate((train_x,test_x),axis = 0)
#     X = np.sum(X,axis = 1,keepdims = True)
    Y = np.concatenate((np.ones(len(train_x)),np.zeros(len(test_x))),axis = 0)
    clf = LogisticRegression().fit(X,Y)
    return clf

def loggingpolicy(x,y):
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression().fit(x,y)
    return clf

In [63]:
def evaluate_ips(logging_index, logging_action, logging_reward,\
                 eval_index, eval_action, eval_reward,\
                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT):
    
    
    
    
#     GT = np.mean(eval_reward)
    ips_list = []
    snips_list = []
    ips_r_list = []
    snips_r_list = []
    
    ips_list_unknown = []
    snips_list_unknown = []
    ips_r_list_unknown = []
    
    
    
    for k in range(1):
#         total_reward = 0
#         for i in range(len(eval_index)):
#             action, prob = take_action(eval_policy[eval_index[i]])
#             if action == label[eval_index[i]]:
#                 total_reward += 1
#         GT = total_reward/len(eval_index)

        
    #     known:

        p_w = my_bound(eval_policy[eval_index, eval_action]/logging_policy[eval_index, eval_action])
        x_w = pdf_eval[eval_index]/pdf[eval_index]

        ips = p_w*eval_reward
        ips_loss = (np.mean(ips) - GT)**2
        ips_list.append(ips_loss)

        snips = p_w*eval_reward/np.mean(p_w)
        snips_loss = (np.mean(snips) - GT)**2
        snips_list.append(snips_loss)

        ips_r = x_w*p_w*eval_reward
        ips_r_loss = (np.mean(ips_r) - GT)**2
        ips_r_list.append(ips_r_loss)
        
        snips_r = x_w*p_w*eval_reward/np.mean(x_w*p_w)
        snips_r_loss = (np.mean(snips_r) - GT)**2
        snips_r_list.append(snips_r_loss)
        
        




#     #     unknown:
#         clf_xshift = p_xshift(xy[logging_index],xy[eval_index])
#         clf_logging = loggingpolicy(xy[logging_index],logging_action)


#         logging_predicted = clf_logging.predict_proba(xy[eval_index])[np.arange(len(eval_index)),np.array(eval_action,np.int)]
#         p_w = eval_policy[eval_reward, eval_action]/logging_predicted

#         xshift_logging = clf_xshift.predict_proba(xy[eval_reward])
#         x_w = xshift_logging[:,1]/xshift_logging[:,0]



#         ips = p_w*eval_reward
#         ips_loss_unknown = (np.mean(ips) - GT)**2
#         ips_list_unknown.append(ips_loss_unknown)

#         snips = p_w*eval_reward/np.mean(p_w)
#         snips_loss_unknown = (np.mean(snips) - GT)**2
#         snips_list_unknown.append(snips_loss_unknown)

#         ips_r = x_w*p_w*eval_reward
#         ips_r_loss_unknown = (np.mean(ips_r) - GT)**2
#         ips_r_list_unknown.append(ips_r_loss_unknown)

    
    return np.mean(ips_list),np.mean(snips_list),np.mean(ips_r_list),np.mean(snips_r_list)
#                 np.mean(ips_list_unknown),np.mean(snips_list_unknown),np.mean(ips_r_list_unknown),
    
    

    
    
    
    
    
    
    
    
    

In [86]:
from ope_v3 import *


class WEIGHT_DATA_SET(data.Dataset):
    '''
    dataset class with instance weight 
    '''
    def __init__(self, data,reward, weights):
        '''
        weights are same dimensional witi original data
        '''
        self.data = torch.tensor(data)
        self.reward = torch.tensor(reward)
        self.weights = torch.tensor(weights)

    def __getitem__(self, index):
        img = self.data[index] 
        target = self.reward[index]
        weight = self.weights[index]
        return img, target, weight

    def __len__(self):
        return len(self.data)

# r is xshift weight
# w is policy shift weight
def train_robust_regression(logging_index, logging_action, logging_reward,\
                 eval_index, eval_action, eval_reward,\
                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                            dm=False,xshift_known = True, policy_known = True,use_xshift = True):
#     print('train models with true density ratios')
    Myy_robust_list = []
    Myx_robust_list = []
    model_robust_list = []
    
    feature = xy[logging_index]
    eval_xshift = pdf_eval[logging_index]
    logging_xshift = pdf[logging_index]
    eval_policy = eval_policy[logging_index,logging_action]
    logging_policy = logging_policy[logging_index,logging_action]
    
    
    for i in range(n_class):
        ## generate training set for action i

        idx = (logging_action == i)


        data_i = feature[idx]
        reward_i = logging_reward[idx]
        eval_xshift_i = eval_xshift[idx]
        logging_xshift_i = logging_xshift[idx]
        eval_policy_i = eval_policy[idx]
        logging_policy_i = logging_policy[idx]
        

        train_size = len(data_i)

        if xshift_known:
            r = logging_xshift_i/eval_xshift_i
        if policy_known:
            w = logging_policy_i/eval_policy_i
        if use_xshift:
            w = r.reshape(len(r)) * w.reshape(len(r))
        
        if dm:
            w = torch.ones(len(data_i))
        
        weight_st = my_bound(w)
        weighted_train = WEIGHT_DATA_SET(data_i,reward_i, weight_st)
#         print(weight_st.shape)

        train_model = Net()

        validate_size = int(0.1*train_size)

        validate_loader = data.DataLoader(data.Subset(weighted_train, range(0, validate_size)),
            batch_size=32, shuffle=True)
        # 10% validation set
        train_loader = data.DataLoader(data.Subset(weighted_train, range(validate_size, train_size)),
            batch_size=32, shuffle=True, )

        train_model, Myy, Myx, _, _, _ = train_validate_test(args,args.lr_robust, 100, "regression", 'cpu', 'False', train_model,
            train_loader, None , validate_loader, n_class , 0.000, d = n_dim,testflag = False)

        model_robust_list.append(train_model)
        Myy_robust_list.append(Myy)
        Myx_robust_list.append(Myx)
    return model_robust_list, Myy_robust_list, Myx_robust_list




In [105]:
def predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                        logging_action,\
                       xy,  pdf, pdf_eval, logging_policy, eval_policy,\
                       dm = False,xshift_known = True, policy_known = True,use_xshift = True):
    
    
    
    if xshift_known:
        r = pdf/pdf_eval
   
        
    if policy_known:
        w = logging_policy/eval_policy
    
    if use_xshift:
        w = r*w
    if dm:
        w = torch.tensor([1])
    
    weight_st = my_bound(w)
    if len(xy.shape) == 2:

        output = model_robust_list[int(logging_action)](torch.tensor(xy))
    else:
        output = model_robust_list[int(logging_action)](torch.tensor(xy).unsqueeze(0))

    Myy = Myy_robust_list[int(logging_action)]
    Myx = Myx_robust_list[int(logging_action)]
    
    
    meanY, varY = ru.predict_regression(torch.tensor(weight_st), Myy, Myx, output, mean0, var0)
    return meanY.detach().numpy()
        
        
        

In [106]:
def evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT,\
                                            model_robust_list, Myy_robust_list, Myx_robust_list,\
                                             dm = False,use_xshift = True):
    
#     model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
#                                             eval_index, eval_action, eval_reward,\
#                                             xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
#                                             dm=dm, use_xshift = use_xshift)
    
#     sample action 
    reg_loss = []
    triple_loss = []
    triple_r_loss = []
    
    r_xshift = pdf_eval[eval_index]/pdf[eval_index]
    sn = np.mean(eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action])
    sn_xshift = np.mean(pdf_eval[eval_index]/pdf[eval_index] * eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action])
    tmp_pred = np.zeros((len(eval_index),n_class))
    for i in range(n_class):
        tmp_eval = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        i,xy[eval_index],  pdf[eval_index], pdf_eval[eval_index], \
                                        logging_policy[eval_index,i], eval_policy[eval_index,i],\
                                        dm = dm,use_xshift = use_xshift)
        tmp_eval = tmp_eval.reshape(-1,len(eval_index))
        tmp_pred[:,i] = tmp_eval
    
    
    
    
    regression_pred = np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
    triple = eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action] * (eval_reward - \
                                    tmp_pred[np.arange(len(eval_index)),eval_action]) + np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
    
    
    
    sn_triple = eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action]/sn * (eval_reward - \
                                    tmp_pred[np.arange(len(eval_index)),eval_action]) + np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
    
    
    
    triple_r = r_xshift * eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action] * (eval_reward - \
                                    tmp_pred[np.arange(len(eval_index)),eval_action]) + np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
    sntriple_r = r_xshift/sn_xshift * eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action]/sn * (eval_reward - \
                                tmp_pred[np.arange(len(eval_index)),eval_action]) + np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)                                                               
    return (np.mean(regression_pred)-GT)**2,(np.mean(triple)-GT)**2,(np.mean(sn_triple)-GT)**2,(np.mean(triple_r)-GT)**2,(np.mean(sntriple_r)-GT)**2



In [107]:
def evaluate_robust(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            GT, dm = False,use_xshift = True):
    
    model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            dm=dm, use_xshift = use_xshift)
    
#     sample action 
    reg_loss = []
    triple_loss = []
    triple_r_loss = []
    sn = np.mean(eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action])
    for t in range(10):
        regression_list = []
        triple_r_list = []
        triple_list = []
        total_reward = 0
        for i in range(len(eval_index)):
            action, prob = take_action(eval_policy[eval_index[i]])
            rew = 0
            if action == label[eval_index[i]]:
                rew = 1
                total_reward += rew

            tmp_eval = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        action,xy[eval_index[i]],  pdf[eval_index[i]], pdf_eval[eval_index[i]], \
                                        logging_policy[eval_index[i],action], prob,\
                                        dm = dm,use_xshift = use_xshift)
            regression_list.append(tmp_eval)
            
            
            tmp_logging = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        eval_action[i],xy[eval_index[i]],  pdf[eval_index[i]], pdf_eval[eval_index[i]], \
                                        logging_policy[eval_index[i],eval_action[i]], eval_policy[eval_index[i],eval_action[i]],\
                                        dm = dm,use_xshift = use_xshift)
            
            

            #     triple
            w = eval_policy[eval_index[i],eval_action[i]]/logging_policy[eval_index[i],eval_action[i]]
            triple_list.append(w*(eval_reward[i] - tmp_logging) + tmp_eval)
            triple_r_list.append(w/sn*(eval_reward[i] - tmp_logging) + tmp_eval)
            
            
            
            rw = my_bound(pdf_eval[eval_index[i]]/pdf[eval_index[i]] * w)
            
            
            
            
#             w = my_bound(eval_policy[eval_index[i],logging_action[i]]/logging_policy[logging_index[i],logging_action[i]])

#             tmp = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
#                                         logging_action[i],xy[logging_index[i]], pdf[logging_index[i]], pdf_eval[logging_index[i]], \
#                                         logging_policy[logging_index[i],logging_action[i]], eval_policy[logging_index[i],logging_action[i]],\
#                                         dm = dm,use_xshift = use_xshift)
            
#             triple_r_list.append(rw*(logging_reward[i] - tmp) + tmp_eval)

#             triple_list.append(w*(logging_reward[i] - tmp) + tmp_eval)


        triple_r = np.mean(triple_r_list)
        triple = np.mean(triple_list)
        robust = np.mean(regression_list)
#         GT = total_reward/len(triple_r_list)
        reg_loss.append(robust)
        triple_loss.append(triple)
        triple_r_loss.append(triple_r)
    
    return (np.mean(reg_loss)-GT)**2,(np.mean(triple_loss)-GT)**2,(np.mean(triple_r_loss)-GT)**2



In [119]:
model_list = [
    'known ips',
    'known snips',
    'known ips-r',
    'known snips-r',
#     'unknown ips',
#     'unknown snips',
#     'unknown ips-r',
    'known dm',
    'known dmdr',
    'known sndmdr',
    'known dmdr_r',
    'known sndmdr_',
    
    
#     'known robust without xshift',
#     'known triple without xshift',
#     'known triple-r without xshift',
    'known robust',
    'known triple',
    'known sntriple',
    'known triple_r',
    'known sntriple_r'
#     'unknown dm',
#     'unknown dmdr',
#     'unknown dmdr-r',
#     'unknown robust without xshift',
#     'unknown triple without xshift',
#     'unknown triple-r without xshift',
#     'unknown robust',
#     'unknown triple',
#     'unknown triple-r'
]
len(model_list)

14

In [109]:
def estimate_GT_with_eval_policy(xy,label,eval_policy):
    eval_policy = eval_policy[train_size:]
    
    xy = xy[train_size:]
    
    label = label[train_size:]
    result = 0
    

    for i in range(len(xy)):
        result += eval_policy[i,label[i]]
    return result/len(eval_policy)
    
    

In [123]:
# ips

from tqdm import tqdm
# logging_a_list = [0.5,0.8,1.2]
# logging_b_list = [0.1,0.2,0.4]
size_list = [500,1000,2000]
logging_policy_list = [[0.9,0.2],[0.7,0.2],[0.5,0.2],[0.1,0]]
# logging_policy_list = [[0.5,0.2],[0.1,0]]


cc = 0    
frame = {}
frame['model'] = model_list
for logging_interval in logging_policy_list:
    cc += 1
    print(cc)
    logging_policy, eval_policy = generate_policy_soften(logging_interval[0],logging_interval[1])

    GT = estimate_GT_with_eval_policy(xy,label,eval_policy)
    ips = []
    snips = []
    ips_r = []
    snips_r = []
    
    
    dm = []
    dmdr = []
    sndmdr = []
    dmdr_r = []
    sndmdr_r = []

#     ips_unknown = []
#     snips_unknown = []
#     ips_r_unknown = []

    robust = []
    triple = []
    sntriple = []
    triple_r = []
    sntriple_r = []
    



#     robust_without_xshift = []
#     triple_without_xshift = []
#     triple_r_without_xshift = []

    pdf, mean_logging, std_logging = get_distribution(fc,logging_a, logging_b)
    pdf_eval, mean_eval, std_eval = get_distribution(fc,eval_a, eval_b,test = True)


    for _ in tqdm(range(20)):

        logging_index, logging_action, logging_reward = \
                sample_logging_data(train_xy, train_label, len(train_xy), pdf, pdf_eval, logging_policy, eval_policy)
        eval_index, eval_action, eval_reward = \
                    sample_eval_data(test_xy, test_label, size, pdf, pdf_eval, logging_policy, eval_policy)  
        model_dm_robust_list, Myy_dm_robust_list, Myx_dm_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
                                        eval_index, eval_action, eval_reward,\
                                        xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                        dm=True, use_xshift = False)
        model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
                                eval_index, eval_action, eval_reward,\
                                xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                dm=False, use_xshift = True)
        for size in size_list:
            eval_index, eval_action, eval_reward = \
                    sample_eval_data(test_xy, test_label, size, pdf, pdf_eval, logging_policy, eval_policy)  

            res = evaluate_ips(logging_index, logging_action, logging_reward,\
                             eval_index, eval_action, eval_reward,\
                             xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT)

            ips.append(res[0])
            snips.append(res[1])
            ips_r.append(res[2])
            snips_r.append(res[3])
            


#             ips_unknown.append(res[3])
#             snips_unknown.append(res[4])



            res = evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                                        eval_index, eval_action, eval_reward,\
                                                        xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT,\
                                                          model_robust_list, Myy_robust_list, Myx_robust_list,\
                                                        dm = False,use_xshift = False)
            robust.append(res[0])
            triple.append(res[1])
            sntriple.append(res[2])
            triple_r.append(res[3])
            sntriple_r.append(res[4])




            res = evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                eval_index, eval_action, eval_reward,\
                                xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT,\
                                model_dm_robust_list, Myy_dm_robust_list, Myx_dm_robust_list,\
                                dm = True,use_xshift = False)
            dm.append(res[0])
            dmdr.append(res[1])
            sndmdr.append(res[2])
            dmdr_r.append(res[3])
            sndmdr_r.append(res[4])
            


            policy_name = str(size) + str(logging_interval)

            result_list = []

            result_list.append(np.sqrt(np.mean(ips)))
            result_list.append(np.sqrt(np.mean(snips)))
            result_list.append(np.sqrt(np.mean(ips_r)))
            result_list.append(np.sqrt(np.mean(snips_r)))

            result_list.append(np.sqrt(np.mean(dm)))
            result_list.append(np.sqrt(np.mean(dmdr)))
            result_list.append(np.sqrt(np.mean(dmdr_r)))
            result_list.append(np.sqrt(np.mean(sndmdr)))
            result_list.append(np.sqrt(np.mean(sndmdr_r)))

            result_list.append(np.sqrt(np.mean(robust)))
            result_list.append(np.sqrt(np.mean(triple)))
            result_list.append(np.sqrt(np.mean(sntriple)))
            result_list.append(np.sqrt(np.mean(triple_r)))
            result_list.append(np.sqrt(np.mean(sntriple_r)))
            
            frame[policy_name] = result_list
frame = pd.DataFrame(frame).round(5)
print(frame)
# frame.to_csv('result2/newadv_noxshift.csv',index = False)


1


100%|███████████████████████████████████████████| 20/20 [03:03<00:00,  9.16s/it]


2


100%|███████████████████████████████████████████| 20/20 [03:05<00:00,  9.26s/it]


3


 85%|████████████████████████████████████▌      | 17/20 [02:44<00:29,  9.68s/it]


KeyboardInterrupt: 

In [120]:
frame['model'] = model_list

In [117]:
len(frame['500[0.5, 0.2]'])

14

In [122]:
frame = pd.DataFrame(frame).round(5)
frame

model  500[0.5, 0.2]  1000[0.5, 0.2]  2000[0.5, 0.2]  \
0          known ips        0.03811         0.03792         0.03792   
1        known snips        0.01577         0.01581         0.01570   
2        known ips-r        2.91269         2.90083         2.89568   
3      known snips-r        0.01577         0.01581         0.01570   
4           known dm        0.06809         0.06755         0.06702   
5         known dmdr        0.01689         0.01677         0.01666   
6       known sndmdr        0.22293         0.22105         0.21922   
7       known dmdr_r        0.01612         0.01600         0.01590   
8      known sndmdr_        0.01590         0.01579         0.01568   
9       known robust        0.07117         0.07085         0.07047   
10      known triple        0.01738         0.01724         0.01710   
11    known sntriple        0.01663         0.01650         0.01636   
12    known triple_r        0.22277         0.22131         0.22002   
13  known sntriple_r        0.01626         0.01613         0.01600   

    500[0.1, 0]  1000[0.1, 0]  2000[0.1, 0]  
0       0.09295       0.09350       0.09289  
1       0.02356       0.02338       0.02322  
2       2.85388       2.84010       2.82925  
3       0.02356       0.02338       0.02322  
4       0.30203       0.30131       0.30063  
5       0.04394       0.04394       0.04357  
6       1.00887       1.00258       0.99789  
7       0.03071       0.03049       0.03031  
8       0.03766       0.03749       0.03745  
9       0.28756       0.28798       0.28850  
10      0.04137       0.04111       0.04077  
11      0.02757       0.02751       0.02739  
12      0.94494       0.94200       0.94008  
13      0.03308       0.03421       0.03436

In [ ]:
frame

In [349]:
for logging_interval in logging_policy_list:
    logging_policy, eval_policy = generate_policy_soften(logging_interval[0],logging_interval[1])
    GT = estimate_GT_with_eval_policy(xy,label,eval_policy)
    GT_train = estimate_GT_with_eval_policy(xy,label,logging_policy)
    print(GT,GT_train)

0.8631166436145842 0.835510213878834
0.8631166436145842 0.650409004956916
0.8631166436145842 0.46613426750536974
0.8631166436145842 0.09999999999999686


In [84]:
logging_policy_list

[[0.9, 0.2], [0.7, 0.2], [0.5, 0.2]]

In [85]:
eval_policy

array([[0.01111111, 0.01111111, 0.01111111, ..., 0.01111111, 0.01111111,
        0.01111111],
       [0.01111111, 0.01111111, 0.01111111, ..., 0.01111111, 0.01111111,
        0.01111111],
       [0.01111111, 0.9       , 0.01111111, ..., 0.01111111, 0.01111111,
        0.01111111],
       ...,
       [0.01111111, 0.01111111, 0.9       , ..., 0.01111111, 0.01111111,
        0.01111111],
       [0.01111111, 0.01111111, 0.01111111, ..., 0.01111111, 0.01111111,
        0.9       ],
       [0.01111111, 0.01111111, 0.01111111, ..., 0.01111111, 0.01111111,
        0.01111111]])

In [86]:
logging_policy

array([[0.05171151, 0.05171151, 0.05171151, ..., 0.05171151, 0.05171151,
        0.05171151],
       [0.04635323, 0.04635323, 0.04635323, ..., 0.04635323, 0.04635323,
        0.04635323],
       [0.06472432, 0.41748114, 0.06472432, ..., 0.06472432, 0.06472432,
        0.06472432],
       ...,
       [0.05340344, 0.05340344, 0.51936907, ..., 0.05340344, 0.05340344,
        0.05340344],
       [0.05631529, 0.05631529, 0.05631529, ..., 0.05631529, 0.05631529,
        0.49316238],
       [0.05912388, 0.05912388, 0.05912388, ..., 0.05912388, 0.05912388,
        0.05912388]])

In [132]:
# ips


# logging_a_list = [0.5,0.8,1.2]
# logging_b_list = [0.1,0.2,0.4]
size_list = [5000,8000]
logging_policy_list = [[0.9,0.2],[0.7,0.2],[0.5,0.2]]
# logging_policy_list = [[0.3,0.2],[0.5,0.2],[0.7,0.2]]

cc = 0    
frame = {}
frame['model'] = model_list
for size in size_list:
    for logging_interval in logging_policy_list:
        cc += 1
        print(cc)
        logging_policy, eval_policy = generate_policy_soften(logging_interval[0],logging_interval[1])
        
        GT = estimate_GT_with_eval_policy(xy,label,eval_policy)
        ips = []
        snips = []
        ips_r = []

        ips_unknown = []
        snips_unknown = []
        ips_r_unknown = []

        robust = []
        triple_noxshift = []
        triple = []

        dm = []
        dmdr_noxshift = []
        dmdr = []

        robust_without_xshift = []
        triple_without_xshift = []
        triple_r_without_xshift = []

        pdf, mean_logging, std_logging = get_distribution(fc,logging_a, logging_b)
        pdf_eval, mean_eval, std_eval = get_distribution(fc,eval_a, eval_b)
        

        for _ in range(100):

            logging_index, logging_action, logging_reward = \
                    sample_logging_data(train_xy, train_label, len(train_xy), pdf, pdf_eval, logging_policy, eval_policy)
            eval_index, eval_action, eval_reward = \
                    sample_eval_data(test_xy, test_label, size, pdf, pdf_eval, logging_policy, eval_policy)  

            res = evaluate_ips(logging_index, logging_action, logging_reward,\
                             eval_index, eval_action, eval_reward,\
                             xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT)

            ips.append(res[0])
            snips.append(res[1])


            ips_unknown.append(res[3])
            snips_unknown.append(res[4])
            
        print(np.sqrt(np.mean(ips)))




1
0.028059157898782626
2
0.01630548792959207
3
0.015008470571984851
4
0.030919175539499875
5


KeyboardInterrupt: 

In [92]:
(np.sqrt(ips))

0.016448142722911653

In [111]:
train_label

array([5, 6, 1, ..., 8, 9, 8])

In [112]:
for i in range(10):
    print(np.sum(train_label == i))

405
397
394
404
394
382
395
396
390
376


In [199]:
logging_policy, eval_policy = generate_policy_soften(0.9,0.2)

GT = estimate_GT_with_eval_policy(xy,label,eval_policy)
logging_policy = logging_policy[train_size:]
eval_policy = eval_policy[train_size:]
result = 0
sim = 100
for t in range(sim):
    size = 1000
    eval_index = np.zeros(len(test_xy),np.int)
    eval_action = np.zeros(len(test_xy),np.int)
    eval_reward = np.zeros(len(test_xy),np.int)
    eval_p = np.zeros(len(test_xy))
    logging_p = np.zeros(len(test_xy))
    index_list = [i for i in range(len(test_xy))]


    
    for i in range(len(test_xy)):
        action, logging_p0 = take_action(logging_policy[i])
        logging_p[i] = logging_p0
        eval_action[i] = action
        eval_p0 = eval_policy[i,action]
        eval_p[i] =  eval_p0

        eval_reward[i] = int(action == test_label[i])
    result += (np.mean(eval_reward*eval_p/logging_p)-GT)**2
print(np.sqrt(result/sim))

0.0868328398315184


In [240]:
logging_policy, eval_policy = generate_policy_soften(0.5,0.2)

GT = estimate_GT_with_eval_policy(xy,label,eval_policy)
logging_policy = logging_policy[train_size:]
eval_policy = eval_policy[train_size:]
result = 0
sim = 2000
for t in range(sim):
    size = 1000
    eval_index = np.zeros(len(test_xy),np.int)
    eval_action = np.zeros(len(test_xy),np.int)
    eval_reward = np.zeros(len(test_xy),np.int)
    eval_p = np.zeros(len(test_xy))
    logging_p = np.zeros(len(test_xy))
    index_list = [i for i in range(len(test_xy))]


    
    for i in range(len(test_xy)):
        action, logging_p0 = take_action(logging_policy[i])
        logging_p[i] = logging_p0
        eval_action[i] = action
        eval_p0 = eval_policy[i,action]
        eval_p[i] =  eval_p0

        eval_reward[i] = int(action == test_label[i])
    result += (np.mean(eval_reward*eval_p/logging_p)-GT)**2
print(np.sqrt(result/sim))

0.029184446708649878


In [241]:
logging_policy, eval_policy = generate_policy_soften(0.7,0.2)

GT = estimate_GT_with_eval_policy(xy,label,eval_policy)
logging_policy = logging_policy[train_size:]
eval_policy = eval_policy[train_size:]
result = 0
sim = 2000
for t in range(sim):
    size = 1000
    eval_index = np.zeros(len(test_xy),np.int)
    eval_action = np.zeros(len(test_xy),np.int)
    eval_reward = np.zeros(len(test_xy),np.int)
    eval_p = np.zeros(len(test_xy))
    logging_p = np.zeros(len(test_xy))
    index_list = [i for i in range(len(test_xy))]


    
    for i in range(len(test_xy)):
        action, logging_p0 = take_action(logging_policy[i])
        logging_p[i] = logging_p0
        eval_action[i] = action
        eval_p0 = eval_policy[i,action]
        eval_p[i] =  eval_p0

        eval_reward[i] = int(action == test_label[i])
    result += (np.mean(eval_reward*eval_p/logging_p)-GT)**2
print(np.sqrt(result/sim))

0.02973519840831019


In [239]:
logging_policy, eval_policy = generate_policy_soften(0.1,0)

GT = estimate_GT_with_eval_policy(xy,label,eval_policy)
logging_policy = logging_policy[train_size:]
eval_policy = eval_policy[train_size:]
result = 0
sim = 500
for t in range(sim):
    size = 1000
    eval_index = np.zeros(len(test_xy),np.int)
    eval_action = np.zeros(len(test_xy),np.int)
    eval_reward = np.zeros(len(test_xy),np.int)
    eval_p = np.zeros(len(test_xy))
    logging_p = np.zeros(len(test_xy))
    index_list = [i for i in range(len(test_xy))]


    
    for i in range(len(test_xy)):
        action, logging_p0 = take_action(logging_policy[i])
        logging_p[i] = logging_p0
        eval_action[i] = action
        eval_p0 = eval_policy[i,action]
        eval_p[i] =  eval_p0

        eval_reward[i] = int(action == test_label[i])
    result += (np.mean(eval_reward*eval_p/logging_p)-GT)**2
print(np.sqrt(result/sim))

0.06465653247907271


In [208]:
max(eval_p/logging_p)

9.0

In [222]:
sum(eval_reward[np.where(eval_p/logging_p == 9.0)])*9

1467

In [226]:
sum(eval_reward[np.where(eval_p/logging_p != 9)]*0.1/9/0.1)

0.6666666666666667

0.006869525126771303